In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
from PIL import Image
# creating image object 
#img = Image.open(r"cat.001") 
  
# using convert method for img1 
#img1 = img.convert("L") 
#img1.show() 

In [3]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode='nearest')

In [6]:
img = load_img('cat_vs_dog_dataset\data\Train\Cats/cat.001.jpg')  # this is a PIL image
#data/train/cats/cat.001.jpg
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150,150,3),padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
#model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))

#model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3),padding='same'))
# the model so far outputs 3D feature maps (height, width, features)

In [9]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [11]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'cat_vs_dog_dataset\data\Train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

Found 2000 images belonging to 2 classes.


In [13]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'cat_vs_dog_dataset\data\Validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 60s 476ms/step - loss: 0.7120 - accuracy: 0.4940 - val_loss: 0.7114 - val_accuracy: 0.5250
Epoch 2/50
125/125 [==============================] - 49s 395ms/step - loss: 0.6972 - accuracy: 0.4900 - val_loss: 0.6854 - val_accuracy: 0.5475
Epoch 3/50
125/125 [==============================] - 47s 378ms/step - loss: 0.6905 - accuracy: 0.5505 - val_loss: 0.6757 - val_accuracy: 0.5400
Epoch 4/50
125/125 [==============================] - 46s 370ms/step - loss: 0.6881 - accuracy: 0.5425 - val_loss: 0.6872 - val_accuracy: 0.5962
Epoch 5/50
125/125 [==============================] - 46s 370ms/step - loss: 0.6821 - accuracy: 0.5575 - val_loss: 0.7023 - val_accuracy: 0.5838
Epoch 6/50
125/125 [==============================] - 46s 370ms/step - loss: 0.6779 - accuracy: 0.5675 - val_loss: 0.6721 - val_accuracy: 0.6175
Epoch 7/50
125/125 [==============================] - 46s 371ms/step - loss: 0.6793 - accuracy: 0.5640 - val_loss: 0.6878 - val_ac